# 📊 Data visualisering og analyse

Denne notebooken bygger videre på data vi har hentet og renset fra både **Frost** og **NILU**. Vi skal nå starte analysen og visualiseringen av dataen og forsøke å trekke ut meningsfull innsikt. Vi vil bruke visuelle fremstillinger for å forstå variasjoner, sammenhenger og trender i miljødataen. Først skal vi se luftkvaliteten i Trondheim før vi går videre på været.

Målet er å identifisere mønstre som er interessante i seg selv (f.eks. hvordan luftkvalitet varierer gjennom året), men også å gi et godt kunnskaps og sammenlignings-grunnlag for å forstå den prediktive analysen i neste steg av prosjektet.

### 🌫️ - Luftkvaliteten i Trondheim
Først skal vi få oversikt over statistiske verdier som gjennomsnitt, median og standaravvik over ulike luftkvalitets indikatorer, som gir oss en rask oversikt over dataens fordeling, variasjon og typiske verdier. Dette hjelper oss med å forstå dataen og identifisere eventuelle avvik eller mønstre.

**DataFrame over luftkvalitet**

In [1]:
# Henter inn liten dataframe for NILU

Videre ønsker vi å se på datasettet i sin helhet. Til dette har vi brukt plotly for å lage en interaktiv graf, hvor man selv enkelt kan filtrere ut enkelte verdier, skalere tidslinjen og lese av nøyaktige verdier for hver observasjon.

In [ ]:
# Hente inn ploty over NILU

Litt om en annen visualiseringen

In [ ]:
# Hente inn visualiseringen nevnt overfor

### 🌦️ - Været i Trondheim
Nå går vi fra luftkvalitet til værdata og skal undersøke været i Trondheim de siste årene. Som med luftkvaliteten starter vi med å analysere statistikken for å få en god oversikt over dataene. 

In [ ]:
# Hente inn dataframe for Frost